# Web Scraping con Requests y BeautifulSoup

Este notebook muestra cómo realizar web scraping usando `requests` y `beautifulsoup4` en el sitio https://quotes.toscrape.com

In [1]:
# Importar las librerías necesarias
import requests
from bs4 import BeautifulSoup
import pandas as pd
from typing import List, Dict
from urllib.parse import urljoin, urlparse
import time

## 1. Obtener el contenido de la página

In [2]:
# URL del sitio a scrapear
url = "https://quotes.toscrape.com"

# Realizar la petición HTTP
response = requests.get(url)

# Verificar que la petición fue exitosa
print(f"Status Code: {response.status_code}")
print(f"Content Length: {len(response.content)} bytes")

# Crear el objeto BeautifulSoup para parsear el HTML
soup = BeautifulSoup(response.content, 'html.parser')
print("\nHTML parseado correctamente!")

Status Code: 200
Content Length: 11064 bytes

HTML parseado correctamente!


### 1.1. Definir variable quotes para uso en ejemplos posteriores

In [3]:
# Definir la variable quotes que se usará en múltiples ejemplos
# Esto asegura que esté disponible en todas las celdas siguientes
quotes = soup.find_all('div', class_='quote')
print(f"Total de quotes encontrados en la página: {len(quotes)}")

Total de quotes encontrados en la página: 10


## 2. Diferentes tipos de selectores en BeautifulSoup

BeautifulSoup ofrece múltiples formas de encontrar elementos en el HTML. A continuación se muestran los diferentes métodos disponibles.

### 2.1. Selector por tag HTML

In [4]:
# Buscar todos los elementos de un tag específico
all_spans = soup.find_all('span')
all_divs = soup.find_all('div')
all_links = soup.find_all('a')

print(f"Total de elementos <span>: {len(all_spans)}")
print(f"Total de elementos <div>: {len(all_divs)}")
print(f"Total de elementos <a>: {len(all_links)}")

# Buscar el primer elemento de un tag
first_span = soup.find('span')
print(f"\nPrimer <span> encontrado: {first_span.get_text()[:50]}...")

Total de elementos <span>: 32
Total de elementos <div>: 28
Total de elementos <a>: 55

Primer <span> encontrado: “The world as we have created it is a process of o...


### 2.2. Selector por clase CSS

In [5]:
# Buscar por clase CSS (método 1: usando class_)
quotes_by_class = soup.find_all('div', class_='quote')
print(f"Quotes encontrados por clase 'quote': {len(quotes_by_class)}")

# Buscar por clase CSS (método 2: usando attrs)
quotes_by_attrs = soup.find_all('div', attrs={'class': 'quote'})
print(f"Quotes encontrados usando attrs: {len(quotes_by_attrs)}")

# Buscar elementos con múltiples clases
# Nota: En este sitio no hay ejemplos, pero así sería:
# elements = soup.find_all('div', class_=['class1', 'class2'])

# Buscar por clase parcial (usando función lambda)
quotes_partial = soup.find_all('div', class_=lambda x: x and 'quote' in x)
print(f"Quotes encontrados con búsqueda parcial: {len(quotes_partial)}")

Quotes encontrados por clase 'quote': 10
Quotes encontrados usando attrs: 10
Quotes encontrados con búsqueda parcial: 10


### 2.3. Selector por ID

In [6]:
# Buscar elemento por ID
# En este sitio no hay muchos IDs, pero así se haría:
# element_by_id = soup.find(id='my-id')
# o
# element_by_id = soup.find('div', id='my-id')

# Buscar todos los elementos que tengan un ID
elements_with_id = soup.find_all(id=True)
print(f"Elementos con atributo ID: {len(elements_with_id)}")

# Mostrar algunos IDs encontrados
print("\nAlgunos IDs encontrados:")
for elem in elements_with_id[:5]:
    print(f"  Tag: {elem.name}, ID: {elem.get('id')}")

Elementos con atributo ID: 0

Algunos IDs encontrados:


### 2.4. Selector por atributos personalizados

In [7]:
# Buscar por atributos específicos
# Ejemplo: buscar todos los links que tengan un atributo href
all_links_with_href = soup.find_all('a', href=True)
print(f"Links con atributo href: {len(all_links_with_href)}")

# Buscar links que contengan un texto específico en el href
author_links = soup.find_all('a', href=lambda x: x and '/author/' in x)
print(f"Links a páginas de autores: {len(author_links)}")

# Mostrar algunos links de autores
print("\nPrimeros 3 links de autores:")
for link in author_links[:3]:
    print(f"  {link.get('href')} - {link.get_text()}")

# Buscar por atributo data-* (si existieran)
# data_elements = soup.find_all(attrs={'data-custom': 'value'})

Links con atributo href: 55
Links a páginas de autores: 10

Primeros 3 links de autores:
  /author/Albert-Einstein - (about)
  /author/J-K-Rowling - (about)
  /author/Albert-Einstein - (about)


### 2.5. Selector CSS (select method)

In [8]:
# Usar selectores CSS con el método select()
# Esto es más similar a jQuery o CSS

# Seleccionar por clase
quotes_css = soup.select('div.quote')
print(f"Quotes usando selector CSS 'div.quote': {len(quotes_css)}")

# Seleccionar por ID (si existiera)
# element = soup.select('#my-id')

# Seleccionar elementos anidados
quote_texts_css = soup.select('div.quote span.text')
print(f"Textos de quotes usando 'div.quote span.text': {len(quote_texts_css)}")

# Seleccionar por atributo
author_links_css = soup.select('a[href*="/author/"]')
print(f"Links de autores usando selector de atributo: {len(author_links_css)}")

# Seleccionar el primer hijo directo
# first_child = soup.select('div > span')

# Seleccionar hermanos adyacentes
# sibling = soup.select('div.quote + div')

# Mostrar el primer texto usando selector CSS
if quote_texts_css:
    print(f"\nPrimer quote usando selector CSS: {quote_texts_css[0].get_text()[:60]}...")

Quotes usando selector CSS 'div.quote': 10
Textos de quotes usando 'div.quote span.text': 10
Links de autores usando selector de atributo: 10

Primer quote usando selector CSS: “The world as we have created it is a process of our thinkin...


### 2.6. Selector por texto contenido

In [9]:
# Buscar elementos que contengan un texto específico
einstein_quotes = soup.find_all('span', class_='text', string=lambda text: text and 'Einstein' in text)
# Nota: esto busca el texto exacto del span, no funciona bien aquí

# Mejor: buscar elementos cuyo texto contenga cierta palabra
all_text_elements = soup.find_all(string=lambda text: text and 'Einstein' in text)
print(f"Elementos de texto que contienen 'Einstein': {len(all_text_elements)}")

# Buscar el elemento padre que contiene cierto texto
for text_elem in all_text_elements[:2]:
    parent = text_elem.parent
    print(f"\nTexto encontrado: {text_elem[:50]}...")
    print(f"Elemento padre: {parent.name} con clase: {parent.get('class')}")

# Buscar por texto exacto
exact_text = soup.find_all('small', class_='author', string='Albert Einstein')
print(f"\nElementos <small> con texto exacto 'Albert Einstein': {len(exact_text)}")

Elementos de texto que contienen 'Einstein': 3

Texto encontrado: Albert Einstein...
Elemento padre: small con clase: ['author']

Texto encontrado: Albert Einstein...
Elemento padre: small con clase: ['author']

Elementos <small> con texto exacto 'Albert Einstein': 3


### 2.7. Selectores combinados y navegación del DOM

In [10]:
# Navegar desde un elemento encontrado
first_quote = soup.find('div', class_='quote')

if first_quote:
    print("Navegación desde el primer quote:")
    
    # Obtener el elemento padre
    parent = first_quote.parent
    print(f"  Elemento padre: {parent.name}")
    
    # Obtener elementos hijos directos
    children = list(first_quote.children)
    print(f"  Número de hijos directos: {len([c for c in children if str(c).strip()])}")
    
    # Obtener todos los descendientes
    descendants = list(first_quote.descendants)
    print(f"  Número de descendientes: {len([d for d in descendants if hasattr(d, 'name')])}")
    
    # Obtener el siguiente hermano
    next_sibling = first_quote.next_sibling
    print(f"  Siguiente hermano existe: {next_sibling is not None}")
    
    # Obtener el hermano anterior
    prev_sibling = first_quote.previous_sibling
    print(f"  Hermano anterior existe: {prev_sibling is not None}")
    
    # Buscar dentro del elemento
    quote_text = first_quote.find('span', class_='text')
    author = first_quote.find('small', class_='author')
    tags = first_quote.find_all('a', class_='tag')
    
    print(f"\n  Contenido del primer quote:")
    print(f"    Texto: {quote_text.get_text()[:50]}...")
    print(f"    Autor: {author.get_text()}")
    print(f"    Tags: {len(tags)} tags encontrados")

Navegación desde el primer quote:
  Elemento padre: div
  Número de hijos directos: 3
  Número de descendientes: 30
  Siguiente hermano existe: True
  Hermano anterior existe: True

  Contenido del primer quote:
    Texto: “The world as we have created it is a process of o...
    Autor: Albert Einstein
    Tags: 4 tags encontrados


## 3. Navegación entre páginas

Para obtener datos de múltiples páginas, necesitamos encontrar el enlace a la siguiente página y hacer una nueva petición HTTP.

**Nota importante:** En este ejemplo usamos `requests` y `BeautifulSoup`, que funcionan bien cuando las páginas se cargan directamente con HTML estático. Si necesitas hacer clic en botones que ejecutan JavaScript o cargan contenido dinámicamente, necesitarías usar herramientas como **Selenium** o **Playwright** para simular la interacción del navegador.

In [11]:
# Función para encontrar el enlace a la siguiente página
def get_next_page_url(soup, base_url):
    """
    Encuentra el enlace a la siguiente página en el HTML parseado.
    
    Args:
        soup: Objeto BeautifulSoup con el HTML parseado
        base_url: URL base para construir URLs relativas
    
    Returns:
        str o None: URL de la siguiente página o None si no existe
    """
    # Buscar el botón/enlace "Next"
    # Opción 1: Buscar por texto del enlace
    next_link = soup.find('a', string=lambda text: text and 'Next' in text)
    
    # Opción 2: Buscar por clase si existe
    if not next_link:
        next_link = soup.find('li', class_='next')
        if next_link:
            next_link = next_link.find('a')
    
    # Opción 3: Buscar cualquier enlace que contenga "next" en su href o texto
    if not next_link:
        next_link = soup.find('a', href=lambda x: x and 'page' in x.lower())
        # Verificar que sea realmente el botón "Next"
        if next_link and 'next' not in next_link.get_text().lower():
            # Buscar el último enlace de paginación (asumiendo que es el "Next")
            pagination_links = soup.find_all('a', href=lambda x: x and '/page/' in x)
            if pagination_links:
                # Ordenar por número de página y tomar el último
                next_link = pagination_links[-1] if len(pagination_links) > 1 else None
    
    if next_link and next_link.get('href'):
        # Construir la URL completa
        next_url = urljoin(base_url, next_link.get('href'))
        return next_url
    
    return None

# Probar la función con la página actual
current_url = "https://quotes.toscrape.com"
next_url = get_next_page_url(soup, current_url)

if next_url:
    print(f"URL de la siguiente página encontrada: {next_url}")
else:
    print("No se encontró enlace a la siguiente página")

URL de la siguiente página encontrada: https://quotes.toscrape.com/page/2/


### 3.1. Obtener datos de la segunda página

In [12]:
# Navegar a la siguiente página y obtener sus datos
if next_url:
    print(f"Accediendo a la página 2: {next_url}\n")
    
    # Hacer petición a la siguiente página
    response_page2 = requests.get(next_url)
    soup_page2 = BeautifulSoup(response_page2.content, 'html.parser')
    
    # Extraer quotes de la segunda página usando diferentes métodos de selección
    print("=" * 80)
    print("QUOTES DE LA PÁGINA 2 (usando diferentes selectores)")
    print("=" * 80)
    
    # Método 1: Por clase CSS tradicional
    quotes_page2_method1 = soup_page2.find_all('div', class_='quote')
    print(f"\nMétodo 1 - find_all('div', class_='quote'): {len(quotes_page2_method1)} quotes")
    
    # Método 2: Usando selector CSS
    quotes_page2_method2 = soup_page2.select('div.quote')
    print(f"Método 2 - select('div.quote'): {len(quotes_page2_method2)} quotes")
    
    # Método 3: Usando attrs
    quotes_page2_method3 = soup_page2.find_all('div', attrs={'class': 'quote'})
    print(f"Método 3 - find_all('div', attrs={{'class': 'quote'}}): {len(quotes_page2_method3)} quotes")
    
    # Mostrar los primeros 3 quotes de la página 2
    print("\n" + "=" * 80)
    print("PRIMEROS 3 QUOTES DE LA PÁGINA 2:")
    print("=" * 80)
    
    for i, quote in enumerate(quotes_page2_method1[:3], 1):
        # Extraer usando diferentes métodos de selección
        quote_text = quote.select_one('span.text').get_text()  # Usando select_one
        author = quote.find('small', class_='author').get_text()  # Usando find
        tags = [tag.get_text() for tag in quote.select('a.tag')]  # Usando select
        
        print(f"\n{i}. {quote_text}")
        print(f"   Autor: {author}")
        print(f"   Tags: {', '.join(tags)}")
        print("-" * 80)
    
    # Comparar autores entre página 1 y página 2
    authors_page1 = [q.find('small', class_='author').get_text() for q in quotes[:5]]
    authors_page2 = [q.find('small', class_='author').get_text() for q in quotes_page2_method1[:5]]
    
    print("\n" + "=" * 80)
    print("COMPARACIÓN DE AUTORES:")
    print("=" * 80)
    print(f"Autores página 1 (primeros 5): {', '.join(authors_page1)}")
    print(f"Autores página 2 (primeros 5): {', '.join(authors_page2)}")
    
else:
    print("No hay siguiente página disponible")

Accediendo a la página 2: https://quotes.toscrape.com/page/2/

QUOTES DE LA PÁGINA 2 (usando diferentes selectores)

Método 1 - find_all('div', class_='quote'): 10 quotes
Método 2 - select('div.quote'): 10 quotes
Método 3 - find_all('div', attrs={'class': 'quote'}): 10 quotes

PRIMEROS 3 QUOTES DE LA PÁGINA 2:

1. “This life is what you make it. No matter what, you're going to mess up sometimes, it's a universal truth. But the good part is you get to decide how you're going to mess it up. Girls will be your friends - they'll act like it anyway. But just remember, some come, some go. The ones that stay with you through everything - they're your true best friends. Don't let go of them. Also remember, sisters make the best friends in the world. As for lovers, well, they'll come and go too. And baby, I hate to say it, most of them - actually pretty much all of them are going to break your heart, but you can't give up because if you give up, you'll never find your soulmate. You'll never fin

### 3.2. Función para scrapear múltiples páginas

### 3.3. Navegación con Selenium (para páginas con JavaScript)

Si necesitas interactuar con páginas que cargan contenido dinámicamente con JavaScript, puedes usar Selenium. Aquí tienes un ejemplo comentado:

In [13]:
# Ejemplo de cómo usar Selenium para hacer click en botones
# NOTA: Requiere instalar selenium: pip install selenium
# También necesitas descargar un driver (ChromeDriver, GeckoDriver, etc.)

"""
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Configurar el driver (ejemplo con Chrome)
# driver = webdriver.Chrome()  # Asegúrate de tener ChromeDriver instalado

# Navegar a la página
# driver.get("https://quotes.toscrape.com")

# Esperar a que el botón "Next" esté disponible y hacer click
# try:
#     next_button = WebDriverWait(driver, 10).until(
#         EC.element_to_be_clickable((By.CSS_SELECTOR, "li.next a"))
#     )
#     next_button.click()
#     
#     # Esperar a que la nueva página cargue
#     time.sleep(2)
#     
#     # Obtener el HTML de la nueva página
#     page_source = driver.page_source
#     soup = BeautifulSoup(page_source, 'html.parser')
#     
#     # Extraer datos de la nueva página
#     quotes = soup.find_all('div', class_='quote')
#     print(f"Quotes encontrados en la página 2: {len(quotes)}")
#     
# finally:
#     driver.quit()
"""

print("Ejemplo de código Selenium (comentado)")
print("Para usar Selenium, descomenta el código arriba y asegúrate de tener:")
print("1. Selenium instalado: pip install selenium")
print("2. Un driver de navegador (ChromeDriver, GeckoDriver, etc.)")
print("3. El driver en tu PATH o especificar la ruta al driver")

Ejemplo de código Selenium (comentado)
Para usar Selenium, descomenta el código arriba y asegúrate de tener:
1. Selenium instalado: pip install selenium
2. Un driver de navegador (ChromeDriver, GeckoDriver, etc.)
3. El driver en tu PATH o especificar la ruta al driver


In [14]:
def scrape_quotes_from_page(soup, page_num=1):
    """
    Extrae todos los quotes de una página parseada.
    
    Args:
        soup: Objeto BeautifulSoup con el HTML parseado
        page_num: Número de página (para logging)
    
    Returns:
        List[Dict]: Lista de diccionarios con información de cada quote
    """
    quotes_data = []
    quotes = soup.find_all('div', class_='quote')
    
    for quote in quotes:
        # Usar diferentes métodos de selección para demostrar variedad
        quote_text = quote.select_one('span.text').get_text()
        author = quote.find('small', class_='author').get_text()
        tags = [tag.get_text() for tag in quote.select('a.tag')]
        
        quotes_data.append({
            'page': page_num,
            'quote': quote_text,
            'author': author,
            'tags': ', '.join(tags),
            'num_tags': len(tags)
        })
    
    return quotes_data

def scrape_multiple_pages(base_url, max_pages=3):
    """
    Scrapea múltiples páginas de quotes.
    
    Args:
        base_url: URL base del sitio
        max_pages: Número máximo de páginas a scrapear
    
    Returns:
        List[Dict]: Lista combinada de todos los quotes de todas las páginas
    """
    all_quotes = []
    current_url = base_url
    page_num = 1
    
    while page_num <= max_pages and current_url:
        print(f"Scrapeando página {page_num}: {current_url}")
        
        # Hacer petición HTTP
        response = requests.get(current_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extraer quotes de esta página
        page_quotes = scrape_quotes_from_page(soup, page_num)
        all_quotes.extend(page_quotes)
        
        print(f"  ✓ {len(page_quotes)} quotes extraídos (Total acumulado: {len(all_quotes)})")
        
        # Buscar siguiente página
        next_url = get_next_page_url(soup, base_url)
        
        if next_url and next_url != current_url:
            current_url = next_url
            page_num += 1
            # Pequeña pausa para ser respetuosos con el servidor
            time.sleep(1)
        else:
            print(f"  No hay más páginas disponibles")
            break
    
    return all_quotes

# Ejemplo: scrapear las primeras 3 páginas
print("Scrapeando múltiples páginas...\n")
all_quotes_multi = scrape_multiple_pages("https://quotes.toscrape.com", max_pages=3)

# Crear DataFrame con todos los quotes
df_multi = pd.DataFrame(all_quotes_multi)

print(f"\n{'=' * 80}")
print(f"RESUMEN: Total de quotes obtenidos de {df_multi['page'].nunique()} páginas: {len(df_multi)}")
print(f"{'=' * 80}")

# Mostrar estadísticas por página
print("\nQuotes por página:")
print(df_multi['page'].value_counts().sort_index())

# Mostrar algunas filas del DataFrame
print(f"\nPrimeras 3 filas del DataFrame combinado:")
display(df_multi.head(3))

Scrapeando múltiples páginas...

Scrapeando página 1: https://quotes.toscrape.com
  ✓ 10 quotes extraídos (Total acumulado: 10)
Scrapeando página 2: https://quotes.toscrape.com/page/2/
  ✓ 10 quotes extraídos (Total acumulado: 20)
Scrapeando página 3: https://quotes.toscrape.com/page/3/
  ✓ 10 quotes extraídos (Total acumulado: 30)

RESUMEN: Total de quotes obtenidos de 3 páginas: 30

Quotes por página:
page
1    10
2    10
3    10
Name: count, dtype: int64

Primeras 3 filas del DataFrame combinado:


,page,quote,author,tags,num_tags
0,1,“The world as we have created it is a process ...,Albert Einstein,"change, deep-thoughts, thinking, world",4
1,1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities, choices",2
2,1,“There are only two ways to live your life. On...,Albert Einstein,"inspirational, life, live, miracle, miracles",5


## 1. Obtener el contenido de la página

In [15]:
# URL del sitio a scrapear
url = "https://quotes.toscrape.com"

# Realizar la petición HTTP
response = requests.get(url)

# Verificar que la petición fue exitosa
print(f"Status Code: {response.status_code}")
print(f"Content Length: {len(response.content)} bytes")

# Crear el objeto BeautifulSoup para parsear el HTML
soup = BeautifulSoup(response.content, 'html.parser')
print("\nHTML parseado correctamente!")

Status Code: 200
Content Length: 11064 bytes

HTML parseado correctamente!


## 2. Listar los primeros 5 quotes

In [16]:
# Encontrar todos los quotes usando diferentes métodos
# Método 1: Por clase CSS (más común)
quotes = soup.find_all('div', class_='quote')

# Métodos alternativos (descomentar para probar):
# Método 2: Usando selector CSS
# quotes = soup.select('div.quote')

# Método 3: Usando attrs
# quotes = soup.find_all('div', attrs={'class': 'quote'})

# Extraer los primeros 5 quotes mostrando diferentes métodos de selección
print("Primeros 5 Quotes (usando diferentes métodos de selección):\n")
print("=" * 80)

for i, quote in enumerate(quotes[:5], 1):
    # Método 1: Usando find con clase
    quote_text_method1 = quote.find('span', class_='text').get_text()
    
    # Método 2: Usando select_one (selector CSS)
    quote_text_method2 = quote.select_one('span.text').get_text()
    
    # Método 3: Usando find con attrs
    quote_text_method3 = quote.find('span', attrs={'class': 'text'}).get_text()
    
    # Método 4: Usando find sin especificar clase (menos específico)
    quote_text_method4 = quote.find('span', {'class': 'text'}).get_text()
    
    # Todos los métodos deberían dar el mismo resultado
    print(f"\n{i}. {quote_text_method1}")
    print(f"   (Métodos usados: find, select_one, attrs - todos equivalentes)")
    print("-" * 80)

Primeros 5 Quotes (usando diferentes métodos de selección):


1. “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
   (Métodos usados: find, select_one, attrs - todos equivalentes)
--------------------------------------------------------------------------------

2. “It is our choices, Harry, that show what we truly are, far more than our abilities.”
   (Métodos usados: find, select_one, attrs - todos equivalentes)
--------------------------------------------------------------------------------

3. “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
   (Métodos usados: find, select_one, attrs - todos equivalentes)
--------------------------------------------------------------------------------

4. “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
   (Métodos usados: find, select_one, attrs

## 3. Listar los autores de los primeros 5 quotes

In [17]:
# Extraer los autores usando diferentes métodos de selección
print("Autores de los primeros 5 quotes:\n")
print("=" * 80)

authors = []
for i, quote in enumerate(quotes[:5], 1):
    # Método 1: Usando find con clase
    author_method1 = quote.find('small', class_='author').get_text()
    
    # Método 2: Usando select_one (selector CSS)
    author_method2 = quote.select_one('small.author').get_text()
    
    # Método 3: Buscar por tag y luego filtrar por clase
    author_method3 = quote.find('small', attrs={'class': 'author'}).get_text()
    
    # Todos los métodos dan el mismo resultado
    author = author_method1
    authors.append(author)
    print(f"{i}. {author} (extraído con find/select_one/attrs)")

print("\n" + "=" * 80)
print(f"\nTotal de autores únicos: {len(set(authors))}")

Autores de los primeros 5 quotes:

1. Albert Einstein (extraído con find/select_one/attrs)
2. J.K. Rowling (extraído con find/select_one/attrs)
3. Albert Einstein (extraído con find/select_one/attrs)
4. Jane Austen (extraído con find/select_one/attrs)
5. Marilyn Monroe (extraído con find/select_one/attrs)


Total de autores únicos: 4


## 4. Listar los tags de los primeros 5 quotes

In [18]:
# Extraer los tags usando diferentes métodos de selección
print("Tags de los primeros 5 quotes:\n")
print("=" * 80)

all_tags = []
for i, quote in enumerate(quotes[:5], 1):
    # Método 1: Usando find_all con clase
    tag_elements_method1 = quote.find_all('a', class_='tag')
    
    # Método 2: Usando select (selector CSS)
    tag_elements_method2 = quote.select('a.tag')
    
    # Método 3: Usando find_all con attrs
    tag_elements_method3 = quote.find_all('a', attrs={'class': 'tag'})
    
    # Método 4: Buscar todos los links y filtrar por clase
    tag_elements_method4 = [a for a in quote.find_all('a') if 'tag' in a.get('class', [])]
    
    # Todos los métodos deberían dar el mismo resultado
    tags = [tag.get_text() for tag in tag_elements_method1]
    all_tags.extend(tags)
    
    print(f"\nQuote {i}:")
    print(f"  Tags: {', '.join(tags)}")
    print(f"  (Extraídos usando find_all/select - {len(tags)} tags)")

print("\n" + "=" * 80)
print(f"\nTotal de tags únicos: {len(set(all_tags))}")
print(f"Tags únicos: {', '.join(sorted(set(all_tags)))}")

Tags de los primeros 5 quotes:


Quote 1:
  Tags: change, deep-thoughts, thinking, world
  (Extraídos usando find_all/select - 4 tags)

Quote 2:
  Tags: abilities, choices
  (Extraídos usando find_all/select - 2 tags)

Quote 3:
  Tags: inspirational, life, live, miracle, miracles
  (Extraídos usando find_all/select - 5 tags)

Quote 4:
  Tags: aliteracy, books, classic, humor
  (Extraídos usando find_all/select - 4 tags)

Quote 5:
  Tags: be-yourself, inspirational
  (Extraídos usando find_all/select - 2 tags)


Total de tags únicos: 16
Tags únicos: abilities, aliteracy, be-yourself, books, change, choices, classic, deep-thoughts, humor, inspirational, life, live, miracle, miracles, thinking, world


## 5. Crear un DataFrame con la información completa

In [19]:
# Crear una lista de diccionarios con la información de los primeros 5 quotes
quotes_data = []

for quote in quotes[:5]:
    quote_text = quote.find('span', class_='text').get_text()
    author = quote.find('small', class_='author').get_text()
    tag_elements = quote.find_all('a', class_='tag')
    tags = [tag.get_text() for tag in tag_elements]
    
    quotes_data.append({
        'quote': quote_text,
        'author': author,
        'tags': ', '.join(tags),
        'num_tags': len(tags)
    })

# Crear el DataFrame
df = pd.DataFrame(quotes_data)

# Mostrar el DataFrame
print("DataFrame con los primeros 5 quotes:\n")
display(df)

DataFrame con los primeros 5 quotes:



,quote,author,tags,num_tags
0,“The world as we have created it is a process ...,Albert Einstein,"change, deep-thoughts, thinking, world",4
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities, choices",2
2,“There are only two ways to live your life. On...,Albert Einstein,"inspirational, life, live, miracle, miracles",5
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"aliteracy, books, classic, humor",4
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"be-yourself, inspirational",2


## 6. Extraer todos los quotes de la página

In [20]:
# Extraer todos los quotes de la página (no solo los primeros 5)
all_quotes_data = []

for quote in quotes:
    quote_text = quote.find('span', class_='text').get_text()
    author = quote.find('small', class_='author').get_text()
    tag_elements = quote.find_all('a', class_='tag')
    tags = [tag.get_text() for tag in tag_elements]
    
    all_quotes_data.append({
        'quote': quote_text,
        'author': author,
        'tags': ', '.join(tags),
        'num_tags': len(tags)
    })

# Crear DataFrame con todos los quotes
df_all = pd.DataFrame(all_quotes_data)

print(f"Total de quotes en la página: {len(df_all)}")
print(f"\nPrimeras 3 filas:\n")
display(df_all.head(3))

Total de quotes en la página: 10

Primeras 3 filas:



,quote,author,tags,num_tags
0,“The world as we have created it is a process ...,Albert Einstein,"change, deep-thoughts, thinking, world",4
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities, choices",2
2,“There are only two ways to live your life. On...,Albert Einstein,"inspirational, life, live, miracle, miracles",5


## 7. Estadísticas básicas

In [21]:
# Estadísticas básicas sobre los quotes
print("Estadísticas de los quotes:\n")
print("=" * 80)

# Número total de quotes
print(f"Total de quotes: {len(df_all)}")

# Número de autores únicos
unique_authors = df_all['author'].unique()
print(f"Número de autores únicos: {len(unique_authors)}")

# Autores más frecuentes
author_counts = df_all['author'].value_counts()
print(f"\nAutores más frecuentes:")
print(author_counts)

# Promedio de tags por quote
avg_tags = df_all['num_tags'].mean()
print(f"\nPromedio de tags por quote: {avg_tags:.2f}")

# Quote con más tags
max_tags_idx = df_all['num_tags'].idxmax()
print(f"\nQuote con más tags ({df_all.loc[max_tags_idx, 'num_tags']} tags):")
print(f"  {df_all.loc[max_tags_idx, 'quote']}")
print(f"  Autor: {df_all.loc[max_tags_idx, 'author']}")

Estadísticas de los quotes:

Total de quotes: 10
Número de autores únicos: 8

Autores más frecuentes:
author
Albert Einstein      3
J.K. Rowling         1
Jane Austen          1
Marilyn Monroe       1
André Gide           1
Thomas A. Edison     1
Eleanor Roosevelt    1
Steve Martin         1
Name: count, dtype: int64

Promedio de tags por quote: 3.00

Quote con más tags (5 tags):
  “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
  Autor: Albert Einstein


## 8. Buscar quotes por autor

In [22]:
# Función para buscar quotes por autor
def get_quotes_by_author(df: pd.DataFrame, author_name: str) -> pd.DataFrame:
    """Retorna todos los quotes de un autor específico"""
    return df[df['author'].str.contains(author_name, case=False)]

# Ejemplo: buscar quotes de Albert Einstein
einstein_quotes = get_quotes_by_author(df_all, "Einstein")

print(f"Quotes de Albert Einstein ({len(einstein_quotes)}):\n")
print("=" * 80)

for idx, row in einstein_quotes.iterrows():
    print(f"\n{row['quote']}")
    print(f"Tags: {row['tags']}")
    print("-" * 80)

Quotes de Albert Einstein (3):


“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Tags: change, deep-thoughts, thinking, world
--------------------------------------------------------------------------------

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Tags: inspirational, life, live, miracle, miracles
--------------------------------------------------------------------------------

“Try not to become a man of success. Rather become a man of value.”
Tags: adulthood, success, value
--------------------------------------------------------------------------------


## 9. Listar todos los tags únicos y su frecuencia

In [23]:
# Extraer todos los tags únicos y contar su frecuencia
from collections import Counter

all_tags_list = []
for tags_str in df_all['tags']:
    if tags_str:
        all_tags_list.extend([tag.strip() for tag in tags_str.split(',')])

# Contar la frecuencia de cada tag
tag_counts = Counter(all_tags_list)

# Mostrar los tags ordenados por frecuencia
print("Tags y su frecuencia:\n")
print("=" * 80)

for tag, count in tag_counts.most_common():
    print(f"{tag}: {count}")

print(f"\nTotal de tags únicos: {len(tag_counts)}")

Tags y su frecuencia:

inspirational: 3
life: 2
humor: 2
change: 1
deep-thoughts: 1
thinking: 1
world: 1
abilities: 1
choices: 1
live: 1
miracle: 1
miracles: 1
aliteracy: 1
books: 1
classic: 1
be-yourself: 1
adulthood: 1
success: 1
value: 1
love: 1
edison: 1
failure: 1
paraphrased: 1
misattributed-eleanor-roosevelt: 1
obvious: 1
simile: 1

Total de tags únicos: 26


## 10. Guardar los datos en un archivo CSV

In [24]:
# Guardar el DataFrame en un archivo CSV
output_file = 'quotes_data.csv'
df_all.to_csv(output_file, index=False, encoding='utf-8')

print(f"Datos guardados en '{output_file}'")
print(f"Total de registros: {len(df_all)}")
print(f"\nPrimeras líneas del archivo:")
display(df_all.head())

Datos guardados en 'quotes_data.csv'
Total de registros: 10

Primeras líneas del archivo:


,quote,author,tags,num_tags
0,“The world as we have created it is a process ...,Albert Einstein,"change, deep-thoughts, thinking, world",4
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities, choices",2
2,“There are only two ways to live your life. On...,Albert Einstein,"inspirational, life, live, miracle, miracles",5
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"aliteracy, books, classic, humor",4
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"be-yourself, inspirational",2
